# **Selenium으로 SNU포털 수강편람 크롤링하기**
마지막 수정일 : 2021년 4월 8일

## **❐ Selenium 소개**

셀레니움(Selenium)은 웹드라이버의 API를 통해 브라우저를 제어하기 때문에, 자바스크립트에 의해 동적으로 생성되는 웹페이지의 데이터를 크롤링 할 때 매우 유용하게 사용되는 크롤링 도구이다.

셀레니움을 사용할 때 가장 보편적인 Chrome driver를 사용한다. 아래의 링크에서 다운받을 수 있다.
https://chromedriver.chromium.org/downloads

셀레니움은 웹드라이버의 DOM에서 요소를 찾을 때 다양한 선택자들 중 골라서 사용하면 된다. 아래와 같이 find_element로 시작하는 함수는 조건에 맞는 요소를 하나만 반환한다. 조건을 만족하는 모든 요소를 반환받고 싶은 경우엔 find_elements로 시작하면 된다.

*   find_element()
*   find_element_by_css_selector()
*   find_element_by_xpath()
*   find_element_by_class_name()
*   find_element_by_tag_name()
*   find_element_by_name()
*   find_element_by_id()
*   find_element_by_link_text()
*   find_element_by_partial_link_text()



## **❐ 필요한 라이브러리 불러오기**

In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import time
import pandas as pd
import csv
import getpass

## **❐ 함수 정의하기**

 - 로그인 처리 함수

In [ ]:
def fn_loginPage(driver, id_, pw_):

    driver.find_element_by_name('userId').send_keys(id_)
    driver.find_element_by_name('password').send_keys(pw_)
    driver.find_element_by_css_selector('#LoginForm > div > fieldset > a.btn_login').click()

 - 수강편람 검색 함수 (콤보박스 처리)

In [ ]:
def fn_search(driver):
    
    Grades = Select(driver.find_element_by_id('srchCptnCorsFg')) #학년 콤보박스
    #Grades.select_by_value('C013300002') #석사 선택
    
    Major = Select(driver.find_element_by_id('srchOpenUpDeptCd')) #학과 콤보박스
    #Major.select_by_value('250') #경영대학 선택

    driver.find_element_by_css_selector('#cond00 > a.btn_search_ok').click()
    
    driver.implicitly_wait(time_to_wait=5) #암묵적으로 웹 로딩을 기다려주는 메소드

 - 검색 결과 수집 함수 (페이지 넘기기 처리)

In [ ]:
def fn_sugang(driver):

    sugang_len = driver.find_element_by_class_name('fc_o').text #검색건수 가져오기
    pages = (int(sugang_len)//10) + 1 #페이지 수 (한 페이지당 10건씩)
    next_pages = pages//10 #페이지바 넘기는 횟수
    
    #column = ['교과구분', '개설대학', '개설학과', '학년', '교과목번호', '강좌번호', '교과목명(부제명)', '학점-강의-실습', '수업교시', '수업형태', '강의실(동-호)', '주담당교수', '강의계획서', '정원', '수강신청인원']
    
    global sugang_list
    
    if pages <= 10: 
        try:
            sugang_list =[]
            for page in range(1,11):
                driver.find_element_by_xpath('//*[@id="content"]/div/div[3]/div[2]/span/a[%s]'%page).click()      
                time.sleep(2)

                table = driver.find_element_by_class_name('tbl_basic')
                tbody = table.find_element_by_tag_name('tbody')
                tr = tbody.find_elements_by_tag_name('tr')
                
                for row in tr:
                    print(row.text.split("\n"))
        
        except:
            print('done')
        
    else:
        
        i=0
        while i <= next_pages: 
            i += 1
            
            try:
                sugang_list =[]
                for page in range(1,11):
                    driver.find_element_by_xpath('//*[@id="content"]/div/div[3]/div[2]/span/a[%s]'%page).click()      
                    time.sleep(2)

                    table = driver.find_element_by_class_name('tbl_basic')
                    tbody = table.find_element_by_tag_name('tbody')
                    tr = tbody.find_elements_by_tag_name('tr')

                    for row in tr:
                        print(row.text.split("\n"))
                        
                driver.find_element_by_xpath('//*[@id="content"]/div/div[3]/div[2]/a[3]').click()
                time.sleep(2)
            
            except:
                print('done')

## ❐ **브라우저에서 크롤링 실행하기**

In [ ]:
chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('headless') #background에서 수행하고자 할 때 사용

driver = webdriver.Chrome(executable_path="/Users/jinsol/Desktop/chromedriver",chrome_options=chrome_options) #크롬드라이버 경로 설정

# 로그인에 필요한 계정정보
id_ = getpass.getpass("아이디를 입력하시오. : ") #getpass를 통해 개인정보를 숨길 수 있음
pw_ = getpass.getpass("패스워드를 입력하시오. : ") 

login_url = "https://my.snu.ac.kr/" #로그인 페이지
driver.get(login_url) # 주소입력하고 enter하는 동작

driver.switch_to_frame('mainFrame')

fn_loginPage(driver, id_, pw_)  # 로그인

driver.find_element_by_xpath('//*[@id="header"]/div[2]/div[1]/ul/li[1]/a').click()

driver.switch_to.window(driver.window_handles[1])
driver.implicitly_wait(time_to_wait=5)

driver.find_element_by_xpath('//*[@id="pgm_container"]/h3[3]').click() #수업/성적 탭
driver.find_element_by_xpath('//*[@id="S0302"]').click() #교과목 탭
driver.find_element_by_xpath('//*[@id="S030201"]').click() #수강편람 탭

time.sleep(5)
driver.switch_to.window(driver.window_handles[2])

fn_search(driver) #검색

fn_sugang(driver) #검색 결과 가져오기

driver.quit()